### Launch the ROS demo

In [1]:
%%bash --bg
trap 'kill $(jobs -p)' EXIT
xterm -e /bin/bash -l -c "roslaunch demo.launch"

### Python Code
Import rospy and initialize a python ROS node

In [2]:
import rospy
rospy.init_node('mynode')

Load the robot model using the `JointStatePublisher`

In [3]:
# joint_state_publisher is not a python package, but a script installed in lib
from robot_model import RobotModel
robot = RobotModel()

Create a publisher and `JointState` message

In [4]:
from sensor_msgs.msg import JointState
pub = rospy.Publisher('/target_joint_states', JointState, queue_size=1)
msg = JointState()

Create a list slider widgets, one for each joint

In [5]:
import ipywidgets
from ipywidgets import FloatSlider, Layout, Button, Box
joint_widgets = [FloatSlider(min = j.min, max = j.max, step = (j.max-j.min) / 100, description = j.name) \
                 for j in robot.active_joints]

Compute forward-kinematics and publish a frame marker

In [6]:
from markers import frame, MarkerArray
marker_pub = rospy.Publisher('/marker_array', MarkerArray, queue_size=1)

def publish_fk_marker():
    T, _ = robot.fk(link='panda_link8', joints={j.description: j.value for j in joint_widgets})
    marker_pub.publish(frame(T))

ImportError: cannot import name 'MarkerArray' from 'markers' (/home/salih/catkin_ws/src/lecture/notebook/markers.py)

React to slider (value) changes by publishing this particular joint

In [7]:
def on_sent(event):
    widget = event.owner
    msg.name = [widget.description]
    msg.position = [widget.value]
    pub.publish(msg)
    #publish_fk_marker()

for widget in joint_widgets:
    widget.observe(on_sent, 'value')

Create a button to randomly generate a pose within joint limits

In [9]:
def set_joints(values):
    for widget, value in values:
        widget.unobserve(on_sent, 'value')
        widget.value = value
        widget.observe(on_sent, 'value')
    msg.name, msg.position = zip(*[(widget.description, widget.value) for widget in joint_widgets])
    pub.publish(msg)
    #publish_fk_marker()

import random
def on_randomize(randomize):
    set_joints([(widget, random.uniform(widget.min, widget.max)) for widget in joint_widgets])

randomize = Button(description='Randomize')       
randomize.on_click(on_randomize)

Create a button to center all joints within their limits

In [10]:
def on_center(b):
    set_joints([(widget, (widget.min + widget.max) / 2) for widget in joint_widgets])

center = Button(description='Center')
center.on_click(on_center)

Collect all widgets (sliders and buttons) in a form and display them

In [11]:
form_items = list(joint_widgets)
form_items += [Box([center, randomize])]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='100%'
))
display(form)

Box(children=(FloatSlider(value=0.0, description='panda_joint1', max=2.9671, min=-2.9671, step=0.059342), Floa…